In [27]:
!pip install streamlit

code = """
import pandas as pd
import plotly.express as px
import streamlit as st
import re
from collections import Counter
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

# Set up page configuration
st.set_page_config(page_title="Medical Conversation Dashboard", page_icon=":hospital:", layout="wide")

# Assuming 'df' is loaded with patient data
@st.cache_data
def load_data():
    # Load the data from your preferred source or file
    df = pd.read_csv("sample_data.csv")  # Replace with your actual path
    df["age"] = pd.to_numeric(df["age"], errors='coerce')  # Convert age to numeric
    return df

df = load_data()

# Sidebar filters
st.sidebar.header("Filter Data:")
age_range = st.sidebar.slider("Select Age Range:", int(df["age"].min()), int(df["age"].max()), (0, 100))
gender = st.sidebar.multiselect("Select Gender:", options=df["gender"].unique(), default=df["gender"].unique())
severity_levels = st.sidebar.multiselect("Select Severity Level:", options=df["Condition_Severity"].unique(), default=df["Condition_Severity"].unique())

# Filter data based on selections
df_selection = df[(df["age"] >= age_range[0]) & (df["age"] <= age_range[1]) & (df["gender"].isin(gender)) & (df["Condition_Severity"].isin(severity_levels))]

# Check if the dataframe is empty:
if df_selection.empty:
    st.warning("No data available based on the current filter settings!")
    st.stop()

# Main page
st.title(":hospital: Medical Conversation Dashboard")
st.markdown("##")

# Optionally, add a horizontal line for separation
st.markdown("---")

# KPIs
total_conversations = len(df_selection)
average_age = round(df_selection["age"].mean(skipna=True), 1)
severity_distribution = df_selection["Condition_Severity"].value_counts()

# Overview Section: Display KPIs
left_column, middle_column, right_column = st.columns(3)
with left_column:
    st.subheader("Total Conversations:")
    st.subheader(f"{total_conversations:,}")
with middle_column:
    st.subheader("Average Age:")
    st.subheader(f"{average_age} years")
with right_column:
    st.subheader("Severity Distribution:")
    for severity, count in severity_distribution.items():
        percentage = (count / total_conversations) * 100
        st.text(f"{severity}: {count} ({percentage:.1f}%)")

st.markdown("---")

# Gender distribution plot
gender_counts = df_selection['gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']  # Rename columns to appropriate labels

gender_plot = px.bar(
    gender_counts,
    x='Gender',
    y='Count',
    labels={'Gender': 'Gender', 'Count': 'Count'},
    title="Gender Distribution",
    color='Gender',  # Use 'Gender' for color distinction
    color_discrete_map={'male': 'lightblue', 'female': 'pink'}
)
gender_plot.update_layout(yaxis_title='Count')  # Update y-axis label

# Age distribution plot

age_plot = px.histogram(df_selection.dropna(subset=['age']), x='age', nbins=30,
                        title="Age Distribution", labels={'age': 'Age'})

# Severity distribution plot

severity_plot = px.bar(severity_distribution, x=severity_distribution.index, y=severity_distribution.values,
                       labels={'x': 'Condition Severity', 'y': 'Count'},
                       title="Condition Severity Distribution",
                       color=severity_distribution.index,
                       color_discrete_map={'High': 'red', 'Medium': 'orange', 'Low': 'yellow', 'Normal': 'lightblue'})

left_column, right_column = st.columns(2)
left_column.plotly_chart(age_plot, use_container_width=True)
right_column.plotly_chart(gender_plot, use_container_width=True)
st.plotly_chart(severity_plot, use_container_width=True)

st.markdown("---")

# Sentiment Analysis Function
def get_sentiment(text):
    if isinstance(text, str):  # Check if text is valid
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    return 0.0

# Apply sentiment analysis on processed conversations
if 'processed_conversation' in df.columns:
    df['conversation_sentiment'] = df['processed_conversation'].apply(get_sentiment)

    # Display Sentiment Polarity Distribution
    st.subheader("Sentiment Polarity Distribution")

    # Histogram plot
    sentiment_plot = px.histogram(
        df,
        x='conversation_sentiment',
        nbins=30,
        labels={'conversation_sentiment': 'Polarity'},
        color_discrete_sequence=['lightgreen']
    )
    sentiment_plot.update_layout(yaxis_title='Frequency')
    st.plotly_chart(sentiment_plot, use_container_width=True)
else:
    st.warning("Processed conversation data not available for sentiment analysis.")

st.markdown("---")

# Map Severity Levels to Numeric Values for Correlation
severity_mapping = {
    "High": 3,
    "Medium": 2,
    "Low": 1,
    "Normal": 0
}

# Map the severity levels in the 'Condition_Severity' column to numeric values
df_selection['Severity_Numeric'] = df_selection['Condition_Severity'].map(severity_mapping)

st.subheader("Sentiment Score by Condition Severity")

# Scatter plot for Severity vs Sentiment
fig, ax = plt.subplots(figsize=(6, 3))
sns.scatterplot(x='Severity_Numeric', y='conversation_sentiment', data=df_selection, hue='Condition_Severity', palette='coolwarm', s=40, ax=ax)
ax.set_ylim(-1.1, 1.1)
plt.xlabel("Condition Severity (Numeric)")
plt.ylabel("Sentiment Score")
plt.title("Sentiment Score by Condition Severity")
plt.legend(title="Severity", loc='upper left')
st.pyplot(fig)

# Optionally, you can compute and display the correlation coefficient
correlation = df_selection['Severity_Numeric'].corr(df_selection['conversation_sentiment'])
st.write(f"Correlation between Severity and Sentiment Score: {correlation:.3f}")

st.markdown("---")

"""
with open('app.py', 'w') as file:
    file.write(code)

In [ ]:
!wget -q -o - ipv4.icanhazip.com

In [28]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.131.140:8501

⠴⠦⠧⠇your url is: https://plain-states-knock.loca.lt
/content/app.py:139: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/content/app.py:139: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/content/app.py:139: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_